In [80]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from tqdm.auto import tqdm, trange
from ipywidgets import IntProgress
import time
from torch import nn
from torch.nn import functional as F

import sys
sys.path.insert(0, '../ml')
from logger import *
from trainer import train
from customlosses import *
# from systemnets import *
from basisfunctions import *

import sys
sys.path.insert(0, '../testproblems/darcy')
from NGO import NGO
from VarMiON import *
from DataModule import *

hparams = {'N_samples': 10000,
           'd': 2,
           'l_min': 0.5,
           'l_max': 1,
           'dtype': torch.float64,
           'precision': 64,
           'devices': [0],
           'loss': weightedrelativeL2,
           'metric': weightedrelativeL2,
           'optimizer': torch.optim.Adam, 
           'learning_rate': 1e-4,
           'batch_size': 100,
           'epochs': 20000,
           'modeltype': 'NGO',
           'data_based': False,
           'gamma_stabilization': 0,
           'quadrature': 'Gauss-Legendre',
           'Q': 100,
           'Q_L': 100,
           'h': [10,10],
           'p': 3,
           'C': 2,
           'n_elements': 10,
           # 'systemnet': UNet,
           'kernel_sizes': [5,5,2,2,2,2,5,5],
           'N_w': 50000,
           'bottleneck_size': 18,
           'NLB_outputactivation': nn.Tanhshrink(),
           'bias_NLBranch': False,
           'bias_LBranch': False,
           'scaling_equivariance': False}

hparams['N'] = np.prod(hparams['h'])
hparams['test_bases'] = [BSplineBasis1D(h=hparams['h'][0], p=hparams['p'], C=hparams['C']), 
                         BSplineBasis1D(h=hparams['h'][1], p=hparams['p'], C=hparams['C'])]
hparams['trial_bases'] = [BSplineBasis1D(h=hparams['h'][0], p=hparams['p'], C=hparams['C']), 
                         BSplineBasis1D(h=hparams['h'][1], p=hparams['p'], C=hparams['C'])]
# hparams['test_bases'] = [ChebyshevTBasis1D(h=hparams['h'][0]), 
#                          ChebyshevTBasis1D(h=hparams['h'][1])]
# hparams['trial_bases'] = [ChebyshevTBasis1D(h=hparams['h'][0]), 
#                          ChebyshevTBasis1D(h=hparams['h'][1])]

params = {}
params['hparams'] = hparams

In [81]:
def balance_N_channels(model, params, N_w):
    count = 0
    print(count)
    i = 1
    N_channels = []
    while count < N_w:
        model.num_channels = i
        model.__init__(params)
        count = sum(p.numel() for p in model.parameters())
        i+=1
        N_channels.append(i)
    return N_channels[-2]

In [82]:
class UNet(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        #Balancing the number of trainable parameters to N_w
        self.num_channels = 1
        count = 0
        num_channels_list = []
        while count < self.hparams['N_w']:
            self.init_layers()
            count = sum(p.numel() for p in self.parameters())
            num_channels_list.append(self.num_channels)
            self.num_channels +=1
        self.num_channels = num_channels_list[-2]
        self.init_layers()
        
    def init_layers(self):
        self.layers = nn.ModuleList()
        self.kernel_sizes = self.hparams['kernel_sizes']
        self.bottleneck_size = self.hparams['bottleneck_size']
        #Layers
        self.layers.append(ReshapeLayer(output_shape=(1,self.hparams['N'],self.hparams['N'])))
        self.layers.append(nn.Conv2d(in_channels=1, out_channels=self.num_channels, kernel_size=self.kernel_sizes[0], stride=self.kernel_sizes[0], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=self.num_channels, kernel_size=self.kernel_sizes[1], stride=self.kernel_sizes[1], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=self.num_channels, kernel_size=self.kernel_sizes[2], stride=self.kernel_sizes[2], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=self.bottleneck_size, kernel_size=self.kernel_sizes[3], stride=self.kernel_sizes[3], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(int(self.bottleneck_size),)))
        self.layers.append(nn.Linear(in_features=int(self.bottleneck_size), out_features=int(self.bottleneck_size)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(ReshapeLayer(output_shape=(self.bottleneck_size,1,1)))
        self.layers.append(nn.ConvTranspose2d(self.bottleneck_size, self.num_channels, kernel_size=self.kernel_sizes[4], stride=self.kernel_sizes[4], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, self.num_channels, kernel_size=self.kernel_sizes[5], stride=self.kernel_sizes[5], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, self.num_channels, kernel_size=self.kernel_sizes[6], stride=self.kernel_sizes[6], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, 1, kernel_size=self.kernel_sizes[7], stride=self.kernel_sizes[7], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(self.hparams['N'],self.hparams['N'])))
        if self.hparams['NLB_outputactivation'] is not None:
            self.layers.append(self.hparams['NLB_outputactivation'])

    def forward(self, x):
        if self.hparams.get('scaling_equivariance',False)==True:
            x_norm = torch.amax(torch.abs(x), dim=(-1,-2))
            x = x/x_norm[:,None,None]
        x0 = x
        x1 = self.layers[0](x0)
        x2 = self.layers[1](x1)
        x3 = self.layers[2](x2)
        x4 = self.layers[3](x3)
        x5 = self.layers[4](x4)
        x6 = self.layers[5](x5)
        x7 = self.layers[6](x6)
        x8 = self.layers[7](x7)
        x9 = self.layers[8](x8)
        x10 = self.layers[9](x9)
        x11 = self.layers[10](x10) + x9
        x12 = self.layers[11](x11)
        x13 = self.layers[12](x12)
        x14 = self.layers[13](x13) + x7
        x15 = self.layers[14](x14)
        x16 = self.layers[15](x15) + x5
        x17 = self.layers[16](x16)
        x18 = self.layers[17](x17) + x3
        x19 = self.layers[18](x18) + x1
        x20 = self.layers[19](x19)
        x21 = self.layers[20](x20)
        y = x21
        if self.hparams.get('scaling_equivariance',False)==True:
            y = y/x_norm[:,None,None]    
        return y


In [83]:
model = UNet(params)

In [84]:
sum(p.numel() for p in model.parameters())

594

In [71]:
model.num_channels

1

In [53]:
x = torch.rand(hparams['batch_size'],hparams['N'],hparams['N'])
model(x)

RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (2 x 2). Kernel size can't be greater than actual input size

In [40]:
y = model(x)
y.shape

torch.Size([100, 64, 64])

In [842]:
class NLBranch_NGO(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        self.layers = nn.ModuleList()
        self.layers.append(ReshapeLayer((hparams['batch_size'],1,hparams['h'],hparams['h'])))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(ReshapeLayer((hparams['batch_size'],64,64)))
        if self.hparams['NLB_outputactivation'] is not None:
            self.layers.append(self.hparams['NLB_outputactivation'])
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        y = x
        return y

In [843]:
model2 = NLBranch_NGO(params)

In [844]:
sum(p.numel() for p in model2.parameters())

21760

In [837]:
x = torch.rand(hparams['batch_size'],hparams['h'],hparams['h'])

In [838]:
model2.forward(x)

tensor([[[ 1.2660e-01,  9.5539e-03,  9.8869e-03,  ...,  1.5795e-01,
           9.2218e-02,  2.5349e-03],
         [ 2.4445e-02,  1.5745e-04,  3.8665e-02,  ...,  1.2233e-03,
           3.9848e-02,  1.7122e-01],
         [ 7.0024e-02,  6.7726e-02,  2.3216e-01,  ...,  6.3851e-02,
           3.4230e-04,  1.7863e-06],
         ...,
         [ 3.6762e-03,  6.2164e-02,  1.4455e-01,  ...,  1.8122e-01,
           9.0918e-02,  2.0407e-01],
         [ 1.1252e-01,  7.3541e-03,  5.1887e-02,  ...,  5.3736e-02,
           1.8951e-01,  8.5818e-02],
         [ 1.2263e-01,  1.2239e-01,  6.7225e-03,  ...,  2.2892e-04,
           2.2715e-01,  8.5528e-03]],

        [[ 8.5735e-03,  2.4894e-04,  1.6277e-01,  ...,  1.9619e-01,
           1.4172e-02,  6.7354e-02],
         [ 2.0241e-02,  3.6662e-02,  5.2415e-03,  ...,  1.3689e-01,
           6.0724e-03,  1.8813e-01],
         [ 6.1002e-07,  1.1386e-01,  3.5557e-03,  ...,  1.1700e-02,
           1.7000e-01,  2.2695e-01],
         ...,
         [ 1.4429e-04,  1